# Homework 8 

Author: Will Eaton
Last updated: Wednesday 30th March 


## Derivations
The derivations here are almost identical to that in the last homework which I provided in my ```Homework7/README.ipynb```. I repeat them below but briefly. 


The 1D wave equation is given as, 
$$ \rho \partial_t^2 s - \partial_x (\mu \partial_x s) = 0 .$$

Starting with the weak form of the equation: 
$$ \int_0^L \rho w\partial_t^2 s dx = -\int_0^L \mu \partial_xw \partial_x s dx + [\mu w \partial_x s]^L_0 $$

where $w$ is a test function. 

Spectral elements in the spatial domain are mapped to a reference element using the mapping 

$$ x(\xi) = \sum_{d=1}^2 N_d(\xi) x_d $$
where $N_d(\xi)$ are linear shape functions in the reference element: 

$$ N_1(\xi) = \frac{1}{2}(1 - \xi)  \: \: \: \: N_2(\xi) = \frac{1}{2}(1 + \xi).$$ 

Hence the geometric mapping has a jacobian that depends on the element width ($h_e$): 

$$ J = \frac{\partial x }{\partial \xi} = \frac{h_e}{2}.$$

Unlike for the geometric mapping, the variable fields (e.g. displacement, velocity, acceleration etc...) are expressed in a basis of Lagrange polynomials of degree $n$ ($l^n$): 

$$ s(x(\xi), t) = \sum_{b = 0}^N s_{b}(t)\:l_b (\xi),$$
$$ w(x(\xi)) = \sum_{a = 0}^N w_{a}\:l_a (\xi).$$


Note here that the discrete sampling points (finite sum over which the field is expressed, $N$) are GLL points. We can express the weak form equation over the whole domain $x \in [0,L]$ as a summation over each of $n_e$ elements, $e$ that can be individually mapped to a reference element. For a single element the weak form becomes: 

$$\int_{-1}^{+1} \rho  w \partial_t^2s \frac{dx}{d\xi} d\xi = -  \int_{-1}^{+1} \mu \partial_{\xi}w \partial_{\xi} T \frac{d\xi}{dx} d\xi  + [\mu w \partial_\xi s \frac{d\xi}{dx}]^L_0 $$

where the last term is the boundary conditions (where the evaluation locations 0, L are still in x domain coordinates). The boundary conditions are actually quite easy to implement here. If there are dirichlet boundary conditions then we can hold the end boundary values in the global domain fixed using: 

```
! NGLOB is the index of the last element in global 1D displacement array
s(1) = 0.0
s(NGLOB) = 0.0 
```

An even-more simple boundary condition is true if we implement a Neumann boundary condition. In this case, $\partial_\xi s \frac{d\xi}{dx}$ vanishes on the boundaries by definition, and so nothing needs to be done.

Subbing in the finite summation approximations for $s$ and $w$ we get: 

$$\sum_{a = 0}^N \sum_{b = 0}^N \int_{-1}^{+1} \rho w_{a}\:l_a l_b J_e \ddot{s}_{b}  d\xi = - \sum_{a = 0}^N \sum_{b = 0}^N  \int_{-1}^{+1} \mu   w_{a} (\partial_{\xi}l_a)  (\partial_{\xi}l_b) s_{b}  J^{-1}_e d\xi  + B $$

where B is the boundary condition. These integrals can be calculated using Gauss quadrature on GLL points: 

$$ \int_{\Omega_e} f(x) dx = \int_{-1}^{+1} f(\xi) J_e d\xi \approx \sum_{\gamma=0}^N W(\xi_{\gamma}) f(\xi_{\gamma}) J(\xi_{\gamma})$$

where $\xi_\gamma$ are the locations discrete GLL points within an element in the referential element domain. Applying these to the above equation for an element we get: 

$$\sum_{a = 0}^N w_{a} \sum_{b = 0}^N \ddot{s}_{b} \sum_{\gamma=0}^N \rho_{\gamma} W_{\gamma} l_a(\xi_{\gamma}) l_b(\xi_{\gamma}) J^{\gamma}_e    = - \sum_{a = 0}^N w_{a}  \sum_{b = 0}^N  s_{b}  \sum_{\gamma=0}^N W_{\gamma} k_{\gamma}   [\partial_{\xi}l_a(\xi_{\gamma}) ]  [\partial_{\xi}l_b(\xi_{\gamma})]  J^{-1}_e  + B $$


Defining a local mass ($M_{ab}$) and stiffness ($K_{ab}$) as 

$$M_{ab} = \sum_{\gamma=0}^N W_{\gamma} \rho_{\gamma} {c_p}_{\gamma}  l_a(\xi_{\gamma}) l_b(\xi_{\gamma}) J^{\gamma}_e$$
$$ K_{ab} = \sum_{\gamma=0}^N W_{\gamma} k_{\gamma}   [\partial_{\xi}l_a(\xi_{\gamma}) ]  [\partial_{\xi}l_b(\xi_{\gamma})]  J^{-1}_e$$

the summation becomes: 

$$\sum_{a = 0}^N \sum_{b = 0}^N w_{a}  M_{ab}\ddot{s}_{b}   = - \sum_{a = 0}^N  \sum_{b = 0}^N  w_{a} K_{ab}s_{b}    + B $$


Importantly, because the discrete sample points ($\xi_{\gamma}$) within each element are GLL points, the Lagrange polynomials have the following property: 
$$ l_a(\xi_i) = \begin{cases}
    1,& i=a\\
    0,              & \text{otherwise}
\end{cases} = \delta_{ia} .$$

Therefore 

$$M_{ab} = \sum_{\gamma=0}^N \rho_{\gamma} {c_p}_{\gamma}  \delta_{ab} J^{\gamma}_e$$
such that $\mathbf{M}$ is a diagonal matrix. It is therefore possible in the time-marching algorithm to treat the mass matrix as a vector. 

